In [55]:
import pandas as pd
import os
import certifi
import requests
from bs4 import BeautifulSoup

from dotenv import load_dotenv
load_dotenv()
KEY = os.getenv("WEATHER_KEY")

In [39]:
url = "https://opendata.cwa.gov.tw/api/v1/rest/datastore/O-A0001-001"
params = {
    "Authorization": KEY,
    "format": "JSON",
}

res = requests.get(url, params=params, verify=False)
res

c:\Users\owner\Desktop\mountain\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'opendata.cwa.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [200]>

In [71]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://hdps.cwa.gov.tw/static/state.html"
res = requests.get(url, verify=False)
res.encoding = 'utf-8'

soup = BeautifulSoup(res.text, "html.parser")

# 抓第一個表格 (現存測站)
table = soup.find("table")
rows = table.find_all("tr")

data = []
for row in rows:
    cols = [td.get_text(strip=True) for td in row.find_all("td")]
    if cols:
        data.append(cols)

columns = ["站號", "站名", "站種", "海拔高度(m)", "經度", "緯度", "城市", "地址", "資料起始日期", "撤站日期", "備註", "原站號", "新站號"]# ✅ 請依實際內容修改

# 建立 DataFrame 並儲存
df = pd.DataFrame(data, columns=columns) 
df = df[df['站號'].str.match(r'^(C0|46)')] # 只要 有人 跟 無人氣象站
df.to_csv("stations.csv", index=False, encoding="utf-8-sig")
print("✅ 轉存完成，共筆數：", len(df))




c:\Users\owner\Desktop\mountain\venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'hdps.cwa.gov.tw'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


✅ 轉存完成，共筆數： 537


In [ ]:
from math import radians, sin, cos, sqrt, atan2

def haversine(lat1, lon1, lat2, lon2):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat/2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

def get_nearest_station_id(lat, lon, df):
    """回傳最近測站的站號"""
    min_dist = float("inf")
    nearest_station_id = None

    for _, row in df.iterrows():
        try:
            station_lat = float(row["緯度"])
            station_lon = float(row["經度"])
            dist = haversine(lat, lon, station_lat, station_lon)
            if dist < min_dist:
                min_dist = dist
                nearest_station_id = row["站號"]
        except ValueError:
            continue

    return nearest_station_id

In [76]:
lat = 24.1367
lon = 124.6858

nearest_id = get_nearest_station_id(lat, lon, df)
nearest_id

'C0U750'